In [9]:
from pymongo import MongoClient
import folium
import pandas as pd
import numpy as np
from folium import plugins
import json
from pandas.io.json import json_normalize


#Calling mongoDB
client = MongoClient()
db = client.db_companies

#Function to print lists
def list_print(var):
    return print(list(var))

#Looking for tags in the db_companies
tags = db.companies.distinct('tag_list')

#Now we look for tags with e-learning.
learning = db.companies.find({'tag_list':{'$regex':'e-learning'}})

#Converting the results to a data frame
df = pd.DataFrame(learning)

#Looking for deadpooled companies in the DataFrame
dead_companies = df.loc[df['deadpooled_year']>0]

#Now we get the offices of the deadpooled companies in another data DataFrame
dead_companies = dead_companies[['name', 'offices']]
dead_companies = dead_companies.dropna()


def get_offices(dataframe):
    offices= []
    hqs= []
    dataframe = dataframe['offices']

    for i in dataframe:
        geoloc = None

        if i[0]['latitude'] and i[0]['longitude']:
            geoloc = {'type':'Point', 'coordinates':[i[0]['longitude'],\
                                                     i[0]['latitude']]}
        hqs.append(geoloc)

        offices.append({'total_offices':len(i), 'latitude':i[0]['latitude'],\
                        'longitude':i[0]['longitude'], 'city':i[0]['city']})

    for i, e in enumerate(hqs):
        if e == None:
            hqs[i]={'type':'none', 'coordinates':['none']}

    return offices, hqs



dc_offices = json_normalize(get_offices(dead_companies)[0])
dc_offices['headquarters'] = get_offices(dead_companies)[1]


#Concatenate the deadpooled DataFrames so we can have just one.
geo_dcoffices = pd.concat([dead_companies, dc_offices], axis=1)[['latitude',\
                                'longitude', 'headquarters', 'total_offices']]

geo_dcoffices = geo_dcoffices.dropna()


#Creating a map to locate the deadpool zones
dead_map = folium.Map(location=[19.41, -99.16], tiles='openstreetmap', zoom_start=12)

#Creating the marker points
points = (geo_dcoffices.latitude, geo_dcoffices.longitude)
coordinates = []
lat = points[0]
long = points[1]

for la, lo in zip(lat,long):
    folium.Marker(location=[la,lo], icon=folium.Icon(icon_color = 'red')).add_to(dead_map)

dead_map

#Now we know where e-learning companies have gone to deadpool

In [31]:
winning_companies = df.loc[(df['number_of_employees']<=15) &(df['founded_year']>=2008)]
winning_companies = winning_companies[['name', 'offices']]
wegotloc = json_normalize(get_offices(dead_companies)[0])
dc_offices['headquarters'] = get_offices(dead_companies)[1]

,name,offices
4,Learn10,"[{'description': None, 'address1': 'Digital Eg..."
6,Supercool School,"[{'description': None, 'address1': '1176 South..."
9,Dojo Learning,"[{'description': 'Dojo HQ', 'address1': '588 A..."
16,Student Teacher Exchange,[]
18,Band Metrics,"[{'description': '', 'address1': 'P.O. Box 250..."
19,Student Teacher Exchange,[]
21,Band Metrics,"[{'description': '', 'address1': 'P.O. Box 250..."
25,Tidalwave Trader,"[{'description': 'Tidalwave Trader, LLC', 'add..."
32,Guaranteach,"[{'description': 'Guaranteach HQ', 'address1':..."
41,Foliospaces,[]
